# 4.9 A senha de tamanho variável

**Objetivo:** Resolver o problema da senha de forma que você não forneça a informação do tamanho da senha para a função que gera a população. Considere que a senha pode ser uma string de 1 até 30 caracteres.

**Dica:** A função objetivo terá que quantificar em sua métrica tanto se o candidato acertou as letras quanto se acertou o tamanho da senha.

**Dica 2:** Você pode criar diferentes estratégias de mutação, não precisa ser apensa uma! Quem sabe uma função de mutação pode alterar letras e a outra pode alterar o tamanho da senha? Ver o exercício “Praticamente um X-man!”.

**Dica 3:** Observe que você terá que pensar um pouco sobre como fará o cruzamento no caso de senhas de tamanhos diferentes. Quem sabe tenha que fazer alguma consideração adicional sobre quais são os valores possíveis para o ponto de corte…
______________________________

# Resolução

Nossa estratégia de resolução se baseou em modificar 4 aspectos principais do *script* relacionado ao problema da senha, descritos a seguir:

1. Criação da população: neste problema, modificamos a função que gera a população (`populacao_senha_s_tamanho`) para gerar populações com tamanhos diferentes de senha usando a função `random.randint` para definir o tamanho da senha.

2. Função objetivo: mudamos a função objetivo para adicionar o valor da diferença entre a senha verdadeira como componente da distância que forma o fitness.

3. Cruzamento: decidimos pelo cruzamento de ponto simples, e adicionamos a verificação para que o máximo do corte (`range`) seja escolhido com base no tamanho da menor senha. Colocamos ainda uma condicional que define que se o tamanho máximo for menor que um, não há cruzamento.

4. Mutação: criamos uma nova função mutação baseada na mutação por salto que muta o tamanho da senha, aumentando ou diminuindo.

Abaixo, as importações relevantes para o algoritmo:

In [2]:
from string import ascii_lowercase, ascii_uppercase, digits

from funcoes_fera_4_9 import populacao_senha_st as cria_populacao
from funcoes_fera_4_9 import funcao_objetivo_pop_senha_st as funcao_objetivo
from funcoes_fera_4_9 import selecao_torneio_min as funcao_selecao
from funcoes_fera_4_9 import cruzamento_ponto_simples_st as funcao_cruzamento
from funcoes_fera_4_9 import mutacao_salto as funcao_mutacao1
from funcoes_fera_4_9 import mutacao_salto_tamanho as funcao_mutacao2

Definimos nossa senha e as variáveis globais relevantes ao problema:

In [5]:
SENHA = list("AmigurumiDeSapo2025")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits
TAMANHO_MAXIMO_SENHA = 30
TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
TAMANHO_TORNEIO = 3

Criamos uma população a partir da quantidade de indivíduos estabelecidos anteriormente. Mostramos também os três primeiros indivíduos dessa população:

In [ ]:
populacao = cria_populacao(TAMANHO_POPULACAO, TAMANHO_MAXIMO_SENHA, CARACTERES_POSSIVEIS)
print(populacao[:3])

[['u', 'm', 'x', 'q', 'p', 'x', 'Q', '0', 'P', '2', 'o', 'K', 'T', 'h', 'Q'], ['6', '8', 'k'], ['P']]


Perceba que há, na população acima, senhas de vários tamanhos. Nossa população é heterogênea, e será ajustada pelo algoritmo a seguir:

In [8]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:

    # Seleção
    fitness = funcao_objetivo(populacao, SENHA)
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)

    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))

    # Encerramento
    populacao = proxima_geracao
    geracao += 1

    fitness = funcao_objetivo(populacao, SENHA)
    menor_fitness_observado = min(fitness)

    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 8
8 E
11 D
37 B
82 A
138 Ani
146 Ami
211 Amig
219 Amigu
278 Amigur
327 Amiguru
389 Amigurum
401 Amigurumi
464 AmigurumiD
473 AmigurumiDe
555 AmigurumiDeS
569 AmigurumiDeSa
592 AmigurumiDeSap
757 AmigurumiDeSapo
787 AmigurumiDeSapo2
799 AmigurumiDeSapo20
912 AmigurumiDeSapo202
925 AmigurumiDeSapo2025


Chegamos em nossa senha definida em 925 gerações! Um número razoável para um problema com um universo de possibilidade maior que $10^7$.

## Considerações finais:

Neste problema, evoluímos um algoritmo genético cujo objetivo era descobrir uma "senha" previamente informada, mas sem saber seu tamanho exato. Fizemos modificações em funções previamente conhecidas de população, *fitness*, mutação e cruzamento.

Da maneira que calculamos o *fitness* e realizamos o cruzamento, em um primeiro momento é natural que a melhor senha tenha 1 caractere, uma vez que sua distância da senha verdadeira será menor. Apesar dessa limitação, o algoritmo é capaz de convergir e não parece diminuir a quantidade de caracteres enquanto não acha o tamanho verdadeiro.

Pode-se dizer que o algoritmo é efetivo para o problema, e que não saber a senha não aumentou muito as gerações até a convergência.